In [ ]:
import numpy as np
from math import sin, cos, exp, sqrt, pi

class Particle:
    def __init__(self, position):
        self.position = position
        self.velocity = None
        self.best_position = position.copy()
        self.best_value = None

In [ ]:
class PSO:
    def __init__(self, function, num_particles, num_dimensions, max_iterations, min_bound, max_bound):
        self.function = function
        self.num_particles = num_particles
        self.num_dimensions = num_dimensions
        self.max_iterations = max_iterations
        self.min_bound = min_bound
        self.max_bound = max_bound
        self.particles = []

        self.global_best_position = None
        self.global_best_value = None

    def optimize(self):
        self._initialize_particles()

        for _ in range(self.max_iterations):
            for particle in self.particles:
                fitness = self.function(*particle.position)

                if particle.best_value is None or fitness > particle.best_value:
                    particle.best_position = particle.position.copy()
                    particle.best_value = fitness

                if self.global_best_value is None or fitness > self.global_best_value:
                    self.global_best_position = particle.position.copy()
                    self.global_best_value = fitness

                particle.velocity = (self._calculate_new_velocity(particle) +
                                     self._calculate_random_velocity())

                particle.position += particle.velocity
                self._apply_bounds(particle)

        return self.global_best_position

    def _initialize_particles(self):
        for _ in range(self.num_particles):
            position = np.random.uniform(self.min_bound, self.max_bound, self.num_dimensions)
            particle = Particle(position)
            particle.velocity = np.zeros(self.num_dimensions)  
            self.particles.append(particle)

    def _calculate_new_velocity(self, particle):
        inertia_weight = 0.5
        cognitive_weight = 2.0
        social_weight = 2.0

        return (inertia_weight * particle.velocity +
                cognitive_weight * np.random.rand() * (particle.best_position - particle.position) +
                social_weight * np.random.rand() * (self.global_best_position - particle.position))

    def _calculate_random_velocity(self):
        random_weight = 0.1
        return random_weight * np.random.uniform(self.min_bound, self.max_bound, self.num_dimensions)

    def _apply_bounds(self, particle):
        particle.position = np.clip(particle.position, self.min_bound, self.max_bound)




In [ ]:
def function(x, y):
    """The mathematical function you want to maximize."""
    return abs(sin(x)*cos(y)*exp(abs(1- (sqrt(x**2+y**2)/pi)))) 

num_particles = 100
num_dimensions = 2
max_iterations = 500
min_bound = -10
max_bound = 10

pso = PSO(function, num_particles, num_dimensions, max_iterations, min_bound, max_bound)
maximum_point = pso.optimize()
maximum_value = function(*maximum_point)

print("Maximum point:", maximum_point)
print("Maximum value:", maximum_value)

Maximum point: [8.05761457 9.66397847]
Maximum value: 19.208432957914336


## Optional Part: Solve TSP with PSO

In [19]:
import numpy as np


class Particle:
    def __init__(self, num_cities):
        self.num_cities = num_cities
        self.position = np.random.permutation(num_cities)
        self.velocity = np.zeros(num_cities)
        self.best_position = self.position.copy()
        self.best_fitness = np.inf


class ParticleSwarmOptimization:
    def __init__(self, num_particles, num_iterations, inertia_weight, cognitive_weight, social_weight):
        self.num_particles = num_particles
        self.num_iterations = num_iterations
        self.inertia_weight = inertia_weight
        self.cognitive_weight = cognitive_weight
        self.social_weight = social_weight

    def solve(self, distance_matrix):
        num_cities = distance_matrix.shape[0]

        particles = []
        global_best_position = None
        global_best_fitness = np.inf

        for _ in range(self.num_particles):
            particle = Particle(num_cities)
            particles.append(particle)

            fitness = self._calculate_fitness(distance_matrix, particle.position)
            if fitness < particle.best_fitness:
                particle.best_position = particle.position.copy()
                particle.best_fitness = fitness

            if fitness < global_best_fitness:
                global_best_position = particle.position.copy()
                global_best_fitness = fitness

        for iteration in range(self.num_iterations):
            for particle in particles:
                self._update_velocity(particle, global_best_position)
                self._update_position(particle)

                fitness = self._calculate_fitness(distance_matrix, particle.position)
                if fitness < particle.best_fitness:
                    particle.best_position = particle.position.copy()
                    particle.best_fitness = fitness

                if fitness < global_best_fitness:
                    global_best_position = particle.position.copy()
                    global_best_fitness = fitness

        return global_best_position, global_best_fitness

    def _update_velocity(self, particle, global_best_position):
        cognitive_component = self.cognitive_weight * np.random.rand() * (particle.best_position - particle.position)
        social_component = self.social_weight * np.random.rand() * (global_best_position - particle.position)
        particle.velocity = self.inertia_weight * particle.velocity + cognitive_component + social_component

    def _update_position(self, particle):
        indices = np.argsort(particle.velocity)
        particle.position = particle.position[indices]

    def _calculate_fitness(self, distance_matrix, path):
        distance = 0
        for i in range(len(path) - 1):
            current_city = path[i]
            next_city = path[i + 1]
            distance += distance_matrix[current_city, next_city]
        return distance


if __name__ == '__main__':


    from math import sqrt
    def fitness(chromosome):
        path=0
        for i in range(len(matrix)):
            gene1 = chromosome[i]
            if(i+1<len(matrix)):
              gene2 = chromosome[i+1]
            else:
              gene2 = chromosome[0]
            path+=matrix[gene1][gene2]
        return path

    f = open('gr229.tsp')
    inp = f.read()
    f.close()
    nodes = [0] * len(inp.split('\n')[7:-2])
    t = 0
    for e in inp.split('\n')[7:-2]:
        nodes[t] = (float(e.split()[1]),float(e.split()[2]))
        t += 1
    matrix = [[0 for i in range(len(nodes))] for j in range(len(nodes))]
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            matrix[i][j] = sqrt((nodes[i][0]-nodes[j][0])**2+(nodes[i][1]-nodes[j][1])**2)

    distance_matrix = np.array(matrix)

    num_particles = 100
    num_iterations = 1000
    inertia_weight = 0.3
    cognitive_weight = 1.9
    social_weight = 0.5

    pso = ParticleSwarmOptimization(num_particles, num_iterations, inertia_weight, cognitive_weight, social_weight)
    best_path, best_distance = pso.solve(distance_matrix)

    print(f"Best path: {best_path}")
    print(f"Best distance: {best_distance}")


Best path: [223 217 228 222 225 227 226 202 188 212 171 219 196 205 209 203 167 193
 215 153 179 174 183 198 176 157 208 135 210 150 132 154 184 191 221 141
 186 139 133 213 160 207 145 156 189 190 130 143 134 220 123 206 195 161
  91 180 159 187 142 204 218 192 168 182 185 122 166  73 131 199 158 216
 163 147 125 126 170 201  80 214 194 152  71 117 111 200  50 107 162 104
 177 116  78 148  75 169 211  87 149  89 120 224  65  33  51  97 181 197
  44 136 114 175  40  53  70  54 124  52  94  74 173 110  77  98 172 103
  86 144 155 165 128  79 108 105 113 109  67 127 119 140 178  14 121 138
 151 146  76  43 102 100  62  85  39 164 129 137   9 106 112  81 118  22
 115  19  99  68  35  66  59  15   2  90  95  96  20  56  93  24  26  16
  37  69  92  36  41  28  82  23  17  55  58  83 101  46  12  88  61  21
   6  84   7   1   0  60   5  31  38  45  47  25  32  72  29  63  27  10
  64  57  42  34  48  18  30  11  49   8   3  13   4]
Best distance: 11252.326602743888


In [40]:
f = open('pr1002.tsp')
inp = f.read()
f.close()
nodes = [0] * len(inp.split('\n')[6:-1])
t = 0
for e in inp.split('\n')[6:-1]:
    nodes[t] = (float(e.split()[1]),float(e.split()[2]))
    t += 1
matrix = [[0 for i in range(len(nodes))] for j in range(len(nodes))]
for i in range(len(nodes)):
    for j in range(len(nodes)):
        matrix[i][j] = sqrt((nodes[i][0]-nodes[j][0])**2+(nodes[i][1]-nodes[j][1])**2)



distance_matrix = np.array(matrix)


num_particles = 50
num_iterations = 1000
inertia_weight = 0.3
cognitive_weight = 1.9
social_weight = 0.5

pso = ParticleSwarmOptimization(num_particles, num_iterations, inertia_weight, cognitive_weight, social_weight)
best_path, best_distance = pso.solve(distance_matrix)

print(f"Best path: {best_path}")
print(f"Best distance: {best_distance}")

ValueError: ignored